### Imports iniciales
Se realizan los siguientes imports:
 - arff para la apertura del archivo de datos
 - pandas para el manejo de datos en tablas
 - numpy para el manejo de operaciones matemáticas y manipulación de matrices, listas y arrays

In [9]:
!pip install qgrid
!pip install scipy
!pip install sklearn

In [10]:
import qgrid


from scipy.io import arff
import pandas as pd
import numpy as np
import scipy as sc


from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


from sklearn.metrics import classification_report


from sklearn.neighbors import KNeighborsClassifier


import numpy.matlib
from scipy.spatial import distance
from scipy.stats import iqr

### Carga y procesamiento de los datos

In [11]:
ds = arff.loadarff('PhishingData.arff')
df = pd.DataFrame(ds[0])
df.head()
dataset = df.to_numpy()
X_parcial = dataset[: , 0:9]
Y_parcial = dataset[: , 9]
X = np.zeros(len(X_parcial))
Y = np.zeros(len(Y_parcial))
j = 0
for i in Y_parcial:
    s = i.decode('UTF-8')
    Y[j] = int(s)
    j = j + 1
X_lista = list()
for i in X_parcial:
    lista = list()
    for k in i:
        s = k.decode('UTF-8')
        lista.append(int(s))
    X_lista.append(lista)
X = np.asarray(X_lista)

### Conteo de los datos

In [12]:
muestras = len(X)
print("Hay " + str(muestras) + " muestras en el conjunto de datos.")
caracteristicas = len(X[0])
print("Hay " + str(caracteristicas) + " características en el conjunto de datos.")

Hay 1353 muestras en el conjunto de datos.
Hay 9 características en el conjunto de datos.


## CLASIFICACIÓN USANDO K - VECINOS

In [13]:
#Función para calcular error de clasificación
def error(Y_lest, Y):
    error = 0
    for ye, y in zip(Y_lest, Y):
        if ye != y:
            error += 1
    error = error/np.size(Y)
    return error


 #####################################################################################


#ETAPA 2: Ejecución de validación cruzada, clasificación k-vecinos
def k_vecinos(n_vec, las_yes):
    folds = 4
    skf = StratifiedKFold(n_splits = folds)
    vec_error_clasif_1 = np.zeros(folds)
    j = 0
    for train_index, test_index in skf.split(X , Y):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]

        #En un sólo pipeline se estandarizan y se procesan los datos con el clasificador_knn
        clasificador_knn = Pipeline([('scaler', StandardScaler()),('knn', KNeighborsClassifier(n_neighbors = n_vec))])
        clasificador_knn.fit(X_train , Y_train)
        Yestim = clasificador_knn.predict(X_test)
        vec_error_clasif_1[j] = error(Yestim, Y_test)
        j = j + 1
        if las_yes == True:
            etiquetas_clasificacion = ['0', '1', '-1']
            print(classification_report(Y_test, Yestim, target_names = etiquetas_clasificacion))
            return
    return np.mean(vec_error_clasif_1) , np.std(vec_error_clasif_1)


 #####################################################################################


# Obtención tabla para determinar número óptimo de vecinos
df_types = pd.DataFrame({'Numero de vecinos' : pd.Series(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '100'])})
df_types["Error_Prueba"] = pd.Series()
df_types["Desviación estándar del error"] = ""

numero_vecinos = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 100]
for i in range(len(numero_vecinos)):
    errorprueba, desvesterror = k_vecinos(numero_vecinos[i], False)
    df_types.loc[i, "Error_Prueba"] = errorprueba # cambiar por el valor correcto
    df_types.loc[i, "Desviación estándar del error"] = desvesterror  # cambiar por el valor correcto

#df_types.loc[1, "Error_Prueba"] = "0.3630" # cambiar por el valor correcto
#df_types.loc[1, "Desviación estándar del error"] = "0.0061"  # cambiar por el valor correcto
#df_types.sort_index(inplace=True)

df_types.set_index(['Numero de vecinos'], inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget.get_changed_df()

C:\Users\Usuario\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,Error_Prueba,Desviación estándar del error
Numero de vecinos,,
1,0.148546,0.00990912
2,0.171451,0.0186221
3,0.146325,0.0181476
4,0.158895,0.020548
5,0.142645,0.00964933
6,0.149282,0.0140693
7,0.144109,0.0125849
8,0.139684,0.00911357
9,0.143380,0.00728688


#### Se concluye que un número óptimo de vecinos para la clasificación es 10. Se tiene el error de prueba más bajo y la desviación estándar del error más baja.

In [14]:
k_vecinos(10, True)

              precision    recall  f1-score   support

           0       0.86      0.93      0.89       176
           1       0.75      0.35      0.47        26
          -1       0.85      0.84      0.84       137

    accuracy                           0.85       339
   macro avg       0.82      0.71      0.74       339
weighted avg       0.85      0.85      0.84       339



## CLASIFICACIÓN USANDO VENTANA DE PARZEN

Primero se estima un ancho de ventana $h$ a partir de la función de ancho de banda de Silverman:

$h = \frac{0.9m}{N^{1/5}} \quad \mbox{with } m = \min\left(\sqrt{\operatorname{Var}(X)},\frac{\operatorname{IQR}(X)}{1.349}\right)$

Donde $X$ representa la muestra (datos de test) Entonces:

#### Se definen primero las funciones necesarias para ejecutar la clasificación basada en un kernel gaussiano para una ventana de Parzen

In [23]:
def kernel_gaussiano(x , h):
    d = len(X[0])
    denominador = ((np.sqrt(2 * np.pi))**d) * (h)**d
    kernel = (np.exp((-0.5)*x**2)) / denominador
    return (kernel)


def Probab_Parzen(X_train, X_test):
    
    
    #Cálculo del ancho de banda de Silverman h, parámetro que equivale al ancho de la ventana o kernel
    s = np.std(X_test)
    iqr = sc.stats.iqr(X_test)
    iqr_a_comparar = iqr / 1.349
    m = min(s, iqr_a_comparar)
    N = len(X_train)
    h = (0.9 * m) / (N)**(1 / 5)
    
    
    Nv = len(X_test) #Se usa en la siguiente linea para determinar la longitud del array que contendrá los y_i = f(x_i)
    Y_test_p = np.zeros(Nv) #Este es f(x*), un array de regresiones obtenidas a partir de x*
    d = len(X[0])
    sumatoria = 0
    
    
    for j in range(Nv):
        for i in range(N):
            distancia = distance.euclidean(X_train[i],X_test[j])
            u_i = distancia / h
            K_u_i = kernel_gaussiano(u_i , h)
            if np.absolute(K_u_i) <= h/2: # ¿h/2?
                sumando = 1
            else:
                sumando = 0
            sumatoria = sumatoria + sumando
        Y_test_p[j] = sumatoria / (N*h**d)
    
    #Debe retornar un array que contenga las predicciones para cada una de las muestras en X_val, en el mismo orden.      
    return Y_test_p

In [24]:
folds = 4
skf = StratifiedKFold(n_splits = folds)
vec_error_clasif_2 = np.zeros(folds)
j = 0
for train_index, test_index in skf.split(X , Y):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]


    #Normalizamos los datos
    media = np.mean(X_train)
    desvia = np.std(X_train)
    X_train = sc.stats.stats.zscore(X_train)
    X_test = (X_test - np.matlib.repmat(media, X_test.shape[0], 1))/np.matlib.repmat(desvia, X_test.shape[0], 1)


    #Se aplica ventana de Parzen
    Y_regres_p = Probab_Parzen(X_train, X_test)
    print(Y_regres_p)
    print("\n\n")
    print(Y_test)
    

    #Se calcula error y desviación estándar del error
    vec_error_clasif_2[j] = error(Y_regres_p , Y_test)
    j = j + 1

error_regres = np.mean(vec_error_clasif_2)
desvest_clasif = np.std(vec_error_clasif_2)
print("El error obtenido es: " + str(error_regres) + " y su desviación es: " + str(desvest_clasif))

[6.08595062e+05 1.21719012e+06 1.82578519e+06 2.43438025e+06
 3.04237512e+06 3.65097018e+06 4.25956524e+06 4.86816030e+06
 5.47555498e+06 6.08354985e+06 6.68854376e+06 7.29713882e+06
 7.90573388e+06 8.51312856e+06 9.12172362e+06 9.73031868e+06
 1.03377134e+07 1.09463084e+07 1.15531029e+07 1.21616980e+07
 1.27702930e+07 1.33782879e+07 1.39856826e+07 1.45942776e+07
 1.52028727e+07 1.58108676e+07 1.64194626e+07 1.70274575e+07
 1.76360526e+07 1.82446476e+07 1.88532427e+07 1.94618377e+07
 2.00704328e+07 2.06790279e+07 2.12870227e+07 2.18956178e+07
 2.25042129e+07 2.31128079e+07 2.37190022e+07 2.43275973e+07
 2.49349920e+07 2.55435870e+07 2.61521821e+07 2.67607771e+07
 2.73693722e+07 2.79755665e+07 2.85823610e+07 2.91903558e+07
 2.97971503e+07 3.04057454e+07 3.10143405e+07 3.16229355e+07
 3.22315306e+07 3.28383251e+07 3.34451195e+07 3.40537146e+07
 3.46623097e+07 3.52703045e+07 3.58788996e+07 3.64868945e+07
 3.70942891e+07 3.77016838e+07 3.83102789e+07 3.89188740e+07
 3.95238679e+07 4.013246

[6.46338128e+05 1.29267626e+06 1.93901439e+06 2.58535251e+06
 3.23169064e+06 3.87802877e+06 4.52373011e+06 5.16943145e+06
 5.81576958e+06 6.46210771e+06 7.10844584e+06 7.75478397e+06
 8.39984852e+06 9.04618665e+06 9.69188799e+06 1.03382261e+07
 1.09845643e+07 1.16302656e+07 1.22766037e+07 1.29229419e+07
 1.35692800e+07 1.42156181e+07 1.48619562e+07 1.55057472e+07
 1.61520853e+07 1.67984235e+07 1.74447616e+07 1.80910997e+07
 1.87374379e+07 1.93837760e+07 2.00301141e+07 2.06764522e+07
 2.13227904e+07 2.19691285e+07 2.26154666e+07 2.32618048e+07
 2.39081429e+07 2.45544810e+07 2.52008191e+07 2.58471573e+07
 2.64896747e+07 2.71360128e+07 2.77823509e+07 2.84274155e+07
 2.90737536e+07 2.97188182e+07 3.03645195e+07 3.10108577e+07
 3.16559222e+07 3.23022603e+07 3.29466881e+07 3.35930262e+07
 3.42342701e+07 3.48799714e+07 3.55263095e+07 3.61720109e+07
 3.68183490e+07 3.74621400e+07 3.81084781e+07 3.87548163e+07
 3.94011544e+07 4.00474925e+07 4.06919203e+07 4.13382584e+07
 4.19845965e+07 4.262902

#### Se llaman las funciones anteriores y se ejecuta la clasificación buscando óptimos

Debido a que hay d = 9 variables y los anchos de las ventanas son muy pequeños, la probabilidad de Parzen se hace muy pequeña con valores de h pequeños.